# 1. Data Pre-processing
### 1.1. Define constants

In [1]:
# DATA_PATH = "/Users/nhantran/Library/CloudStorage/GoogleDrive-little.tea.07@gmail.com/My Drive/Workspace/VinBigData/gsm/customer-insights"
DATA_PATH = "/Users/nhantran/Library/CloudStorage/GoogleDrive-little.tea.07@gmail.com/My Drive/Workspace/VinBigData/gsm/customer-insights"

SAMPLE_BOOKING_GSM = "data_booking-gsm_part_0.csv"
SAMPLE_BOOKING_PARTNER = "data_booking-partner_part_0.csv"
SAMPLE_CUSTOMER = "data_customer_part_0.csv"

CHARTSTUDIO_USERNAME = "panicpotatoe"
CHARTSTUDIO_API_KEY = "dUH29yoiNEtXmp251Eh6"

### 1.2. Import libraries and settings

In [2]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

### 1.3. Import data and apply data types

In [3]:
df = pd.read_csv(f"{DATA_PATH}/{SAMPLE_BOOKING_GSM}")
df.sample(1)

/var/folders/ry/rm14stxj66n9l_hyf7tdmwlh0000gn/T/ipykernel_34414/1029334747.py:1: DtypeWarning: Columns (58,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{DATA_PATH}/{SAMPLE_BOOKING_GSM}")


,booking_id,customer_id,driver_id,service_id,price_id,vehicle_id,reason_id,priority_id,payment_id,promotion_id,app_id,user_id,`type`,start_address,end_address,start_point,waypoints,list_driver,meter_switch,estimated_time,status,distance,note,type_charged,round_trip,cancel,other_reason,license_plate,total_price,discount,rate_price,is_rush,count_call,customer_app_version,customer_phone_name,customer_phone_os,driver_app_version,driver_phone_name,driver_phone_os,currency,queued,deleted,payment_gateway,journey_status,paid_surcharge,add_customer,add_data,pickup_fee,start_time,accept_time,cancel_time,booking_time,end_request,promotion_code_id,payment_transaction_id,bookingcol,leads_status,business_code,business_note,did_change_destination,payment_code,promotion_code,promotion_session_id,message_error,is_assist,passenger_name,passenger_phone,customer_card_id,distance_source
16259,12314202772162007,23041802772,GSM-230401238,1,5,557.0,39.0,0,11,NaN,1.0,NaN,1,"3b P. Phan Đình Phùng, Hàng Mã, Ba Đình, Hà Nộ...","44a P. Hàng Chuối, Phạm Đình Hổ, Hoàn Kiếm, Hà...",21.039794921875 105.84650758840682,"[{""text"":""3b P. Phan Đình Phùng, Hàng Mã, Ba Đ...",NaN,0,11.5,0,3.656,NaN,1,0,1,NaN,NaN,65000,0,1.0,0,1,NaN,NaN,NaN,10020.0,coconut - Active 3 - vsmart,"Android 10, API 29",₫,0,0,NaN,5,0,NaN,NaN,0,1688134787000,1.688135e+12,1.688135e+12,NaN,NaN,NaN,GSM12314202772162007,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Define crucial functions
### 2.1. Define functions to extract location information from the address

In [4]:
def get_address_level(address):
    # create blank variables to store address in 1 line of code
    unit = street = ward = district = province = country = str()

    # split start address to get country, province, district, ward, street, unit
    try:
        country = address.split(",")[-1]
    except:
        pass
    try:
        province = address.split(",")[-2]
    except:
        pass
    try:
        district = address.split(",")[-3]
    except:
        pass
    try:
        ward = address.split(",")[-4]
    except:
        pass
    try:
        street = address.split(",")[-5]
    except:
        pass
    try:
        unit = address.split(",")[-6]
    except:
        pass

    return unit, street, ward, district, province, country


# address_level = get_address_level(address)
# print(
#     f"unit: {address_level[0]}\nstreet: {address_level[1]}\nward: {address_level[2]}\ndistrict: {address_level[3]}\nprovince: {address_level[4]}\ncountry: {address_level[5]}"
# )

### 2.2. Define functions to extract convert Unix time to datetime

In [5]:
# create a function that get Unix time and return datetime
def get_datetime(unix_time, to_string=True, is_millisecond=True):
    import datetime

    # define a null timestamp variable with data type is datetime
    timestamp = None

    # check if unix_time is not null
    if not pd.isnull(unix_time):
        # check if input is millisecond or second
        if is_millisecond:
            timestamp = datetime.datetime.fromtimestamp(unix_time / 1000)
        else:
            timestamp = datetime.datetime.fromtimestamp(unix_time)

        # convert datetime to string if to_string is True
        if to_string:
            timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")
    else:
        timestamp = None
    return timestamp

# 3. Data Pre-processing

In [6]:
# convert all Unix timestamp to datetime using get_datetime function
df["start_time"] = df["start_time"].apply(
    get_datetime, to_string=False, is_millisecond=True
)
df["accept_time"] = df["accept_time"].apply(
    get_datetime, to_string=False, is_millisecond=True
)
df["cancel_time"] = df["cancel_time"].apply(
    get_datetime, to_string=False, is_millisecond=True
)
df["booking_time"] = df["booking_time"].apply(
    get_datetime, to_string=False, is_millisecond=True
)
df.dtypes

booking_id                         int64
customer_id                       object
driver_id                         object
service_id                         int64
price_id                           int64
vehicle_id                       float64
reason_id                        float64
priority_id                        int64
payment_id                         int64
promotion_id                     float64
app_id                           float64
user_id                          float64
`type`                             int64
start_address                     object
end_address                       object
start_point                       object
waypoints                         object
list_driver                       object
meter_switch                       int64
estimated_time                   float64
status                             int64
distance                         float64
note                              object
type_charged                       int64
round_trip      

# 3. Data analysis
## 3.1. Data analysis by pick up and drop off locations

In [7]:
# create new df with `start_address` and `end_address` columns
# df_address = df[["start_time", "start_address", "end_address"]].copy()
df_address = df.copy()

# get address level of start address and end address
start_address_level = df_address["start_address"].apply(get_address_level)
end_address_level = df_address["end_address"].apply(get_address_level)

# add new columns for start address unit, street, ward, district, province, country
df_address.loc[:, "start_address_l5"] = start_address_level.apply(
    lambda x: x[0]
).str.strip()
df_address.loc[:, "start_address_l4"] = start_address_level.apply(
    lambda x: x[1]
).str.strip()
df_address.loc[:, "start_address_l3"] = start_address_level.apply(
    lambda x: x[2]
).str.strip()
df_address.loc[:, "start_address_l2"] = start_address_level.apply(
    lambda x: x[3]
).str.strip()
df_address.loc[:, "start_address_l1"] = start_address_level.apply(
    lambda x: x[4]
).str.strip()
df_address.loc[:, "start_address_l0"] = start_address_level.apply(
    lambda x: x[5]
).str.strip()

# add new columns for end address unit, street, ward, district, province, country
df_address.loc[:, "end_address_l5"] = end_address_level.apply(
    lambda x: x[0]
).str.strip()
df_address.loc[:, "end_address_l4"] = end_address_level.apply(
    lambda x: x[1]
).str.strip()
df_address.loc[:, "end_address_l3"] = end_address_level.apply(
    lambda x: x[2]
).str.strip()
df_address.loc[:, "end_address_l2"] = end_address_level.apply(
    lambda x: x[3]
).str.strip()
df_address.loc[:, "end_address_l1"] = end_address_level.apply(
    lambda x: x[4]
).str.strip()
df_address.loc[:, "end_address_l0"] = end_address_level.apply(
    lambda x: x[5]
).str.strip()

df_address.to_csv(f"processed-data/data_booking-gsm_part_0_address.csv", index=False)
df_address.sample(5)

,booking_id,customer_id,driver_id,service_id,price_id,vehicle_id,reason_id,priority_id,payment_id,promotion_id,app_id,user_id,`type`,start_address,end_address,start_point,waypoints,list_driver,meter_switch,estimated_time,status,distance,note,type_charged,round_trip,cancel,other_reason,license_plate,total_price,discount,rate_price,is_rush,count_call,customer_app_version,customer_phone_name,customer_phone_os,driver_app_version,driver_phone_name,driver_phone_os,currency,queued,deleted,payment_gateway,journey_status,paid_surcharge,add_customer,add_data,pickup_fee,start_time,accept_time,cancel_time,booking_time,end_request,promotion_code_id,payment_transaction_id,bookingcol,leads_status,business_code,business_note,did_change_destination,payment_code,promotion_code,promotion_session_id,message_error,is_assist,passenger_name,passenger_phone,customer_card_id,distance_source,start_address_l5,start_address_l4,start_address_l3,start_address_l2,start_address_l1,start_address_l0,end_address_l5,end_address_l4,end_address_l3,end_address_l2,end_address_l1,end_address_l0
12879,12317204067162007,23051704067,NaN,49,61,NaN,NaN,0,20,NaN,1.0,NaN,1,"8VM4+QWH, Ganh Dau, Phú Quốc, Kien Giang, Vietnam","6XQ8+72J, Dương Đông, Phú Quốc, Kien Giang, Vi...",10.335405754768345 103.85734671726823,"[{""text"":""8VM4+QWH, Ganh Dau, Phú Quốc"",""geoco...","[""PQC-230624044""]",0,23.3,-1,17.943,NaN,1,0,0,NaN,NaN,296000,0,1.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,₫,0,0,NaN,0,0,NaN,NaN,0,2023-06-30 20:17:31,NaT,NaT,None,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,8VM4+QWH,Ganh Dau,Phú Quốc,Kien Giang,Vietnam,,6XQ8+72J,Dương Đông,Phú Quốc,Kien Giang,Vietnam
736,12317909857162001,23052409857,NaN,7,52,NaN,NaN,0,1,NaN,1.0,NaN,1,"PMPF+HM4, Đ. 783 Tạ Quang Bửu, Bình Hưng, Quận...","53 Tản Đà, Phường 10 (Quận 5), Quận 5, Thành p...",10.73626708984375 106.67430711929136,"[{""text"":""PMPF+HM4, Đ. 783 Tạ Quang Bửu, Bình ...",NaN,0,8.1,-1,3.063,NaN,1,0,0,NaN,NaN,56000,0,1.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,₫,0,0,NaN,0,0,NaN,NaN,0,2023-06-30 18:09:57,NaT,NaT,None,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PMPF+HM4,Đ. 783 Tạ Quang Bửu,Bình Hưng,Quận 8,Thành phố Hồ Chí Minh,Vietnam,,53 Tản Đà,Phường 10 (Quận 5),Quận 5,Thành phố Hồ Chí Minh,Việt Nam
15481,4234940116249453,C-23030101,HUE-230517299,17,19,1577.0,NaN,1,1,NaN,NaN,NaN,4,"Giảng đường Đại học sư phạm, Lê Lợi, Phú Hội, ...","FJ78+9M Thành phố Huế, Thừa Thiên Huế, Vietnam",16.4688621 107.59245,"[{""x"":""16.4688621"",""y"":""107.59245"",""text"":""Giả...",NaN,1,0.0,2,0.000,NaN,2,0,0,NaN,NaN,0,0,1.0,0,1,NaN,NaN,NaN,10019.0,jacaranda - Aris - vsmart,"Android 11, API 30",₫,0,0,NaN,0,0,NaN,NaN,0,2023-06-30 21:03:20,NaT,NaT,None,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Giảng đường Đại học sư phạm,Lê Lợi,Phú Hội,Thành phố Huế,Thừa Thiên Huế,Vietnam,,,,FJ78+9M Thành phố Huế,Thừa Thiên Huế,Vietnam
2578,12320602875162020,23062002875,NaN,46,48,NaN,NaN,0,1,NaN,1.0,NaN,1,"Oscar Saigon Hotel, Đường Nguyễn Huệ, Bến Nghé...","Cari Ấn Độ Musa, Đường Sư Vạn Hạnh, Phường 9 (...",10.7748014 106.7033222,"[{""text"":""Oscar Saigon Hotel, Đường Nguyễn Huệ...",NaN,0,15.7,-1,4.561,NaN,1,0,0,NaN,NaN,86000,0,1.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,₫,0,0,NaN,0,0,NaN,NaN,0,2023-06-30 18:27:38,NaT,NaT,None,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oscar Saigon Hotel,Đường Nguyễn Huệ,Bến Nghé,Quận 1,Thành phố Hồ Chí Minh,Việt Nam,Cari Ấn Độ Musa,Đường Sư Vạn Hạnh,Phường 9 (Quận 5),Quận 5,Thành phố Hồ Chí Minh,Việt Nam
10183,12318505683162002,23053005683,HUE-230606020,16,18,2997.0,NaN,0,1,NaN,1.0,NaN,1,"1 kiet 43 Yết Kiêu, Thuận Hoà, Thành phố Huế, ...","6A Võ Thị Sáu, Phú Hội, Thành phố Huế, Thừa Th...",16.467559814453125 107.56951143708089,"[{""text"":""1 kiet 43 Yết Kiêu, Thuận Hoà, Thành...",NaN,0,10.1,2,3.854,NaN,1,0,0,NaN,NaN,64000,0,1.0,0,1,NaN,NaN,NaN,10741.0,iPhone,iOS 16.3.1,₫,0,0,NaN,5,0,NaN,NaN,0,2023-06-30 19:26:56,2023-06-30 19:27:04,NaT,None,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,

In [8]:
# replace all null values with "no location"
df_address = df_address.fillna("no location")
# replace all blank values with "no location"
df_address = df_address.replace("", "no location")

In [9]:
# FILTER
# filter data with `start_address_l1` contains `Hà Nội`
df_address_by_province = df_address[
    df_address["start_address_l1"].str.contains("Hà Nội", na=False)
].copy()

# FILTER
# filter data with `start_time` is within 2023-06-30 18:00:00 and 2023-06-30 19:00:00
# df_address_by_province = df_address_by_province[
#     (df_address_by_province["start_time"] >= "2023-06-30 18:00:00")
#     & (df_address_by_province["start_time"] <= "2023-06-30 19:00:00")
# ].copy()

# get min and max `start_time`
min_start_time = df_address_by_province["start_time"].min()
max_start_time = ["start_time"].max()
print(f"min_start_time: {min_start_time}\nmax_start_time: {max_start_time}")

min_start_time: 2023-06-30 18:00:00
max_start_time: 2023-06-30 21:28:33


In [15]:
len(df_address_by_province)

2519

### 3.1.1. Sankey digram by administritive area level 3

In [10]:
# create a sample dataframe from `df_address` including `start_address_l3`, `end_address_l3`
df_address_sample = df_address_by_province[
    ["start_address_l3", "end_address_l3"]
].reset_index(drop=True)

# OPTIONAL
# drop rows that have `start_address_l3` and `end_address_l3` start with numbers
df_address_sample = df_address_sample[
    ~df_address_sample["start_address_l3"].str.contains("^\d", na=False)
]
df_address_sample = df_address_sample[
    ~df_address_sample["end_address_l3"].str.contains("^\d", na=False)
]

# create a list of wards based on `start_address_l3` and `end_address_l3`
wards = list(
    set(df_address_sample["start_address_l3"].unique())
    | set(df_address_sample["end_address_l3"].unique())
)
wards = sorted(wards)

print(wards[0])

An Khánh


In [11]:
# label encoding for `start_address_l3` and `end_address_l3`
# by replacing `start_address_l3` and `end_address_l3` with index of `ward`
df_address_sample.loc[:, "start_address_l3"] = df_address_sample[
    "start_address_l3"
].apply(lambda x: wards.index(x))
df_address_sample.loc[:, "end_address_l3"] = df_address_sample["end_address_l3"].apply(
    lambda x: wards.index(x)
)
df_address_sample.head(5)

,start_address_l3,end_address_l3
0,115,35
1,61,137
2,110,372
3,385,384
4,246,360


In [12]:
# group `df_address_sample` by `start_address_ward` and `end_address_ward` and count the number of rows as `count`

df_address_sample_sk = df_address_sample.groupby(
    ["start_address_l3", "end_address_l3"]
).size()

# reset index of `df_address_sample_sk` and rename columns
df_address_sample_sk = df_address_sample_sk.reset_index()
df_address_sample_sk.columns = ["start_address_l3", "end_address_l3", "count"]

# FILTER
# filter `df_address_sample_sk` with `count` > 100
df_address_sample_sk = df_address_sample_sk[df_address_sample_sk["count"] > 2]

df_address_sample_sk.sample(5)

,start_address_l3,end_address_l3,count
638,128,19,3
1533,364,122,6
188,36,121,5
6,0,272,3
553,121,35,4


In [13]:
# create a plotly sankey diagram from wloc_df_pivot_nom_from_workplace_encoded
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

# create sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                # label=["cat", "dog", "domestic", "female", "male", "wild"],
                label=wards,
                # color="blue",
            ),
            link=dict(
                source=df_address_sample_sk["start_address_l3"],
                target=df_address_sample_sk["end_address_l3"],
                value=df_address_sample_sk["count"],
            ),
        )
    ]
)
fig.update_layout(
    title_text=("Sankey Diagram - Administrative level 2 - Hanoi"),
    font_size=10,
    width=2000,
    height=2000,
)
fig.show()

# save sankey diagram to html file
fig.write_html("3.1.1_sankey_l3.html")

# save the plotly sankey diagram as html file to plotly
tls.set_credentials_file(
    username=CHARTSTUDIO_USERNAME,
    api_key=CHARTSTUDIO_API_KEY,
)
public_link = py.plot(
    fig,
    filename="3.1.1_sankey_l3.html",
    auto_open=False,
)
print(f"published at: {public_link}")

published at: https://plotly.com/~panicpotatoe/40/


### 1.1. Sankey digram by administritive area level 3

In [14]:
# create a sample dataframe from `df_address` including `start_address_l2`, `end_address_l2`
df_address_sample = df_address_by_province[
    ["start_address_l2", "end_address_l2"]
].reset_index(drop=True)

# OPTIONAL
# drop rows that have `start_address_l2` and `end_address_l2` start with numbers
df_address_sample = df_address_sample[
    ~df_address_sample["start_address_l2"].str.contains("^\d", na=False)
]
df_address_sample = df_address_sample[
    ~df_address_sample["end_address_l2"].str.contains("^\d", na=False)
]

# create a list of wards based on `start_address_l2` and `end_address_l2`
wards = list(
    set(df_address_sample["start_address_l2"].unique())
    | set(df_address_sample["end_address_l2"].unique())
)
wards = sorted(wards)

print(wards[0])
# label encoding for `start_address_l2` and `end_address_l2`
# by replacing `start_address_l2` and `end_address_l2` with index of `ward`
df_address_sample.loc[:, "start_address_l2"] = df_address_sample[
    "start_address_l2"
].apply(lambda x: wards.index(x))
df_address_sample.loc[:, "end_address_l2"] = df_address_sample["end_address_l2"].apply(
    lambda x: wards.index(x)
)
df_address_sample.head(5)
# group `df_address_sample` by `start_address_ward` and `end_address_ward` and count the number of rows as `count`

df_address_sample_sk = df_address_sample.groupby(
    ["start_address_l2", "end_address_l2"]
).size()

# reset index of `df_address_sample_sk` and rename columns
df_address_sample_sk = df_address_sample_sk.reset_index()
df_address_sample_sk.columns = ["start_address_l2", "end_address_l2", "count"]

# OPTIONAL
# filter `df_address_sample_sk` with `count` > 100
df_address_sample_sk = df_address_sample_sk[df_address_sample_sk["count"] > 1]

df_address_sample_sk.sample(5)
import plotly.graph_objects as go

# create sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                # label=["cat", "dog", "domestic", "female", "male", "wild"],
                label=wards,
                # color="blue",
            ),
            link=dict(
                source=df_address_sample_sk["start_address_l2"],
                target=df_address_sample_sk["end_address_l2"],
                value=df_address_sample_sk["count"],
            ),
        )
    ]
)
fig.update_layout(
    title_text=("Sankey Diagram - Administrative level 2 - Hanoi"),
    font_size=10,
    width=2000,
    height=2000,
)
fig.show()

# save sankey diagram to html file
fig.write_html("3.1.2_sankey_l2.html")

# save the plotly sankey diagram as html file to plotly
tls.set_credentials_file(
    username=CHARTSTUDIO_USERNAME,
    api_key=CHARTSTUDIO_API_KEY,
)
public_link = py.plot(
    fig,
    filename="3.1.2_sankey_l2.html",
    auto_open=False,
)
print(f"published at: {public_link}")

Ba Dinh district


published at: https://plotly.com/~panicpotatoe/42/
